## An example workflow

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from pyPoseidon.model import *
from pyPoseidon.dem import *
from pyPoseidon.meteo import *
import datetime
from pyPoseidon.utils.data import *

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs


hv.notebook_extension('bokeh')

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':160., # lat/lon window
     'maxlon':179.,
     'minlat':-49.,
     'maxlat':-33.,
     'solver':'d3d',
     'resolution':0.2, #grid resoltuion 
     'step':20, # step for output of map field in d3d
     'start_date':'2013-10-28',
     'time_frame':'2d',
     'ncores': 4 , #number of cores
     'rpath':'./test/', #location of calc folder
     'conda_env':'mpi2' # optional conda env for running the solver
    }

### Create model

In [ ]:
#initialize a model
b = model(**dic)

### set it up

In [ ]:
b.set() #set it up 

### output to rpath 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

### Visualise

In [ ]:
otp = data([dic['rpath']])

#### Grid & Bathymetry 

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

otp.dem.bathymetry.plot.pcolormesh('longitude', 'latitude', ax=ax);

gp = ax.scatter(otp.grid.lons, otp.grid.lats, s=2, color='k',  label='grid points', transform=ccrs.PlateCarree());

pp = ax.scatter(otp.xh, otp.yh, s=2, color='b', label='pressure points', transform=ccrs.PlateCarree());

ax.legend()

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

#### Variable snapshot

In [ ]:
otp.vars.S1[10,:,:].plot() 

#### Holoviews

In [ ]:
%%opts Image [width=650 height=450 colorbar=True] (cmap='jet')
#Fast but no map
otp.hview('S1').to(hv.Image, ['XZ','YZ'])#.hist()

#### Geoviews

In [ ]:
#with map and zooming but dynamic colorbar to make it fast, otherwise slow and heavy
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above"  tools=['hover']] (cmap='viridis')
otp.gview('S1').to(gv.Image, ['XZ','YZ'], dynamic=True) * gf.coastline() * gf.borders()

#### Inherent Animation option

In [ ]:
# relatively fast and you can save in mp4
otp.frames(['S1'],title='SSH')

In [ ]:
# relatively fast and you can save in mp4
otp.frames(['U1','V1'],title='Sea Vectors',scale=.1)

### Observation points

In [ ]:
otp.obs.locations # in-situ measurements available within the lat/lon window

In [ ]:
# Get with index number
p = otp.obs.iloc(1)
p.head()

In [ ]:
#get with Station Name  
p = otp.obs.loc('Gisborne')
p.head()

In [ ]:
#plot
ax = p.plot()

#### sample nearest point from simulation 

In [ ]:
lat,lon = otp.obs.locations.loc[1,['Latitude','Longitude']] #get lat lon from tide gauge
print lat, lon

In [ ]:
ts = otp.vars.S1.sel(XZ=[lon], method='nearest').sel(YZ=[lat], method='nearest')
otp.vars.XZ.sel(XZ=[lon], method='nearest'), otp.vars.YZ.sel(YZ=[lat], method='nearest')

In [ ]:
#Check ts, could be nan
ts = ts.squeeze().to_pandas()
ts.head()

In [ ]:
ts.plot()

In [ ]:
## Join the graphs
ax = ts.plot(figsize=(10,8),color=['r'],label='d3d')
p.plot(ax=ax)
ax.legend(loc='center left',bbox_to_anchor=(1.1, 0.5))